In [9]:
from pyvi import ViTokenizer
#import ViTokenizer

In [10]:
def sent2labels(sent):
    return [label for token, label in sent]

In [11]:
def gen_tag(word):
    syllables = word.split('_')
    if not any(syllables):
        return [(word, 'B_W')]
    else:
        output = [(syllables[0], 'B_W')]
        for item in syllables[1:]:
            output.append((item, 'I_W'))
    return output

vtb = open('../tokenize_dataset/vtb.txt')
vtb_data = []
for line in vtb:
    # print(line)
    tokens = [token.split('/')[0] for token in line.split(' ')]
    sentence = []
    for token in tokens:
        sentence.extend(gen_tag(token))
    vtb_data.append(sentence)
vtb.close()    

In [12]:
vtb_sentences = [ViTokenizer.ViTokenizer.sent2features(sent, True) for sent in vtb_data]
vtb_labels = [sent2labels(sent) for sent in vtb_data]

In [32]:
#vtb_sentences[1]
#vtb_labels[1]
vlsp2016_train_sentences = [ViTokenizer.ViTokenizer.sent2features(sent, True) for sent in vlsp2016_train]

In [33]:
vlsp2016_dev_sentences = [ViTokenizer.ViTokenizer.sent2features(sent, True) for sent in vlsp2016_dev]

In [34]:
vlsp2016_train_labels = [sent2labels(sent) for sent in vlsp2016_train]
vlsp2016_dev_labels = [sent2labels(sent) for sent in vlsp2016_dev]

In [41]:
vlsp2018_train_sentences = [ViTokenizer.ViTokenizer.sent2features(sent, True) for sent in vlsp2018_train]
vlsp2018_dev_sentences = [ViTokenizer.ViTokenizer.sent2features(sent, True) for sent in vlsp2018_test]

In [42]:
vlsp2018_train_labels = [sent2labels(sent) for sent in vlsp2018_train]
vlsp2018_dev_labels = [sent2labels(sent) for sent in vlsp2018_test]

3205

In [13]:
from sklearn.model_selection import train_test_split

In [14]:
X_vtb_train, X_vtb_test, y_vtb_train, y_vtb_test = train_test_split(vtb_sentences, vtb_labels, test_size=0.33, random_state=42)

In [51]:
X_train = X_vtb_train + vlsp2016_train_sentences + vlsp2018_train_sentences
X_test = X_vtb_test + vlsp2016_dev_sentences + vlsp2018_dev_sentences
y_train = y_vtb_train + vlsp2016_train_labels + vlsp2018_train_labels
y_test = y_vtb_test + vlsp2016_dev_labels + vlsp2018_dev_labels

In [53]:
assert len(X_train) == len(y_train)
assert len(X_test) == len (y_test)

In [54]:
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)
crf.fit(X_train, y_train)

CRF(algorithm='lbfgs', all_possible_states=None, all_possible_transitions=True,
    averaging=None, c=None, c1=0.1, c2=0.1, calibration_candidates=None,
    calibration_eta=None, calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=False)

In [55]:
labels = list(crf.classes_)
labels

['B_W', 'I_W']

In [56]:
y_pred = crf.predict(X_test)
metrics.flat_f1_score(y_test, y_pred,
                      average='weighted', labels=labels)

0.9850910262608653

In [57]:
print(metrics.flat_classification_report(
    y_test, y_pred, labels=labels, digits=3
))

              precision    recall  f1-score   support

         B_W      0.989     0.993     0.991    247802
         I_W      0.970     0.952     0.961     58847

    accuracy                          0.985    306649
   macro avg      0.979     0.972     0.976    306649
weighted avg      0.985     0.985     0.985    306649



In [58]:
import scipy.stats
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV

crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    max_iterations=100,
    all_possible_transitions=True
)
params_space = {
    'c1': scipy.stats.expon(scale=0.5),
    'c2': scipy.stats.expon(scale=0.05),
}

# use the same metric for evaluation
f1_scorer = make_scorer(metrics.flat_f1_score,
                        average='weighted', labels=labels)

# search
rs = RandomizedSearchCV(crf, params_space,
                        cv=5,
                        verbose=1,
                        n_jobs=-1,
                        n_iter=50,
                        scoring=f1_scorer)
rs.fit(X_train, y_train)

Fitting 5 folds for each of 50 candidates, totalling 250 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


KeyboardInterrupt: 

In [11]:
def gen_tag(word, separator=' '):
    syllables = word.split(separator)
    if not any(syllables):
        return [(word, 'B_W')]
    else:
        output = [(syllables[0], 'B_W')]
        for item in syllables[1:]:
            output.append((item, 'I_W'))
    return output

In [12]:
def read_vlsp2016_format(filename):
    vlsp2016 = open(filename, 'r')
    vlsp2016_data = []
    sentence = []
    for line in vlsp2016:
        line = line.strip()

        if line:
            word = line.split('\t')[0]
            sentence.extend(gen_tag(word, ' '))
        else:
            vlsp2016_data.append(sentence)
            sentence = []
    return vlsp2016_data

In [13]:
vlsp2016_train = read_vlsp2016_format('../tokenize_dataset/vlsp2016/train.txt')

In [14]:
vlsp2016_dev = read_vlsp2016_format('../tokenize_dataset/vlsp2016/dev.txt')

In [16]:
len(vlsp2016_train)

14860

In [ ]:
import ViTokenizer
ViTokenizer.ViTokenizer.sylabelize('Trung_tá Huỳnh_Trung_Phong-Trưởng phòng CSGT đường_bộ-đường_sắt')

In [17]:
import glob

In [18]:
vlsp2018_train_files = glob.glob('../tokenize_dataset/tokenized_vlsp2018/VLSP2018-NER-train-Jan14/*/*')

In [19]:
vlsp2018_test_files = glob.glob('../tokenize_dataset/tokenized_vlsp2018/VLSP2018-NER-dev/*/*')

In [24]:
import json
def read_vlsp2018_format(files):
    sentence_set = []
    for file in files:
        with open(file, 'r') as f:
            for line in f:
                sents = json.loads(line)['tokenized']
                for i in sents:
                    tokens = []
                    # print(sents[i])
                    _, words = ViTokenizer.ViTokenizer.sylabelize(sents[i])
                    # print(words)
                    for word in words:
                        tokens.extend(gen_tag(word, '_'))
                    sentence_set.append(tokens)
    return sentence_set

In [39]:
vlsp2018_test = read_vlsp2018_format(vlsp2018_test_files)

In [40]:
vlsp2018_train = read_vlsp2018_format(vlsp2018_train_files)

In [28]:
len(test_sentences)

3205

In [29]:
len(train_sentences)

9493

In [31]:
train_sentences[-100]

[('Khi', 'B_W'),
 ('mua', 'B_W'),
 ('đậu', 'B_W'),
 ('phụ', 'I_W'),
 ('phải', 'B_W'),
 ('chú', 'B_W'),
 ('ý', 'I_W'),
 ('đến', 'B_W'),
 ('bìa', 'B_W'),
 ('của', 'B_W'),
 ('miếng', 'B_W'),
 ('đậu', 'B_W'),
 (',', 'B_W'),
 ('nếu', 'B_W'),
 ('bìa', 'B_W'),
 ('miếng', 'B_W'),
 ('đậu', 'B_W'),
 ('cứng', 'B_W'),
 ('khi', 'B_W'),
 ('tiếp', 'B_W'),
 ('xúc', 'I_W'),
 ('với', 'B_W'),
 ('không', 'B_W'),
 ('khí', 'I_W'),
 ('sẽ', 'B_W'),
 ('bị', 'B_W'),
 ('ôxy', 'B_W'),
 ('hóa', 'I_W'),
 ('chuyển', 'B_W'),
 ('sang', 'B_W'),
 ('màu', 'B_W'),
 ('vàng', 'B_W'),
 ('.', 'B_W'),
 ('Nhìn', 'B_W'),
 ('miếng', 'B_W'),
 ('đậu', 'B_W'),
 ('phụ', 'I_W'),
 ('càng', 'B_W'),
 ('vàng', 'B_W'),
 ('nhiều', 'B_W'),
 ('thì', 'B_W'),
 ('đậu', 'B_W'),
 ('phụ', 'I_W'),
 ('đó', 'B_W'),
 ('chứa', 'B_W'),
 ('thạch', 'B_W'),
 ('cao', 'I_W'),
 ('càng', 'B_W'),
 ('nhiều', 'B_W'),
 ('.', 'B_W')]

In [47]:
a = [1, 2, 3]
b = [4, 5]
c = [6, 7]

In [48]:
a + b + c

[1, 2, 3, 4, 5, 6, 7]

In [60]:
import pickle
# Its important to use binary mode 
tokenized_X_train = open('tokenized_X_train.pkl', 'ab') 
tokenized_X_test = open('tokenized_X_test.pkl', 'ab') 
tokenized_y_train = open('tokenized_y_train.pkl', 'ab')
tokenized_y_test = open('tokenized_y_test.pkl', 'ab') 

pickle.dump(X_train, tokenized_X_train)                      
tokenized_X_train.close() 
pickle.dump(X_test, tokenized_X_test)                      
tokenized_X_test.close() 
pickle.dump(y_train, tokenized_y_train)                      
tokenized_y_train.close() 
pickle.dump(y_test, tokenized_y_test)                      
tokenized_y_test.close() 

In [1]:
import pickle

In [6]:
crf = pickle.load(open('./tokenizer_model_py3.pkl', 'rb'))

In [7]:
crf

CRF(algorithm='lbfgs', all_possible_states=None, all_possible_transitions=True,
    averaging=None, c=None, c1=0.801450912928389, c2=0.13865320826084276,
    calibration_candidates=None, calibration_eta=None,
    calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=False)

In [16]:
labels = list(crf.classes_)
labels
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

In [17]:
y_pred = crf.predict(X_vtb_test)
metrics.flat_f1_score(y_vtb_test, y_pred,
                      average='weighted', labels=labels)

0.9828069906496412

In [19]:
print(metrics.flat_classification_report(
    y_vtb_test, y_pred, labels=labels, digits=3
))

              precision    recall  f1-score   support

         B_W      0.987     0.993     0.990     77070
         I_W      0.961     0.928     0.944     14244

    accuracy                          0.983     91314
   macro avg      0.974     0.961     0.967     91314
weighted avg      0.983     0.983     0.983     91314



In [20]:
tokenized_X_train = open('tokenized_X_train.pkl', 'rb') 
tokenized_X_test = open('tokenized_X_test.pkl', 'rb') 
tokenized_y_train = open('tokenized_y_train.pkl', 'rb')
tokenized_y_test = open('tokenized_y_test.pkl', 'rb') 

X_train = pickle.load(tokenized_X_train)                      
X_test = pickle.load(tokenized_X_test)

y_train = pickle.load(tokenized_y_train)
y_test = pickle.load(tokenized_y_test)
                      
tokenized_X_train.close() 
tokenized_X_test.close()
tokenized_y_train.close() 
tokenized_y_test.close() 

In [21]:
y_pred = crf.predict(X_test)
metrics.flat_f1_score(y_test, y_pred,
                      average='weighted', labels=labels)

0.9854075980136232

In [22]:
print(metrics.flat_classification_report(
    y_test, y_pred, labels=labels, digits=3
))

              precision    recall  f1-score   support

         B_W      0.989     0.993     0.991    247802
         I_W      0.971     0.952     0.962     58847

    accuracy                          0.985    306649
   macro avg      0.980     0.973     0.976    306649
weighted avg      0.985     0.985     0.985    306649

